<a href="https://colab.research.google.com/github/koconno8/CS436FinalProject/blob/main/FinalProject436.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
from torchvision import datasets, transforms
import numpy as np
import time
import random
import getopt
import sys
import os
import logging
from torch.autograd import Variable

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)
        self.relu = nn.ReLU(inplace=False)  # Add explicit ReLU module

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x) if self.shortcut else x
        out = self.relu(out)
        return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        # Define ReLU modules with inplace=False
        self.relu = nn.ReLU(inplace=False)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        # Use explicit ReLU module instead of functional
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) if self.shortcut else x
        out = self.relu(out)  # Final activation
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = conv3x3(3,64)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=False)  # Add explicit ReLU module
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Replace F.avg_pool2d with nn.AdaptiveAvgPool2d
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))  # Use module instead of functional
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])

def test_resnet():
    net = ResNet50()
    y = net(Variable(torch.randn(1,3,32,32)))
    print(y.size())

In [ ]:
os.makedirs('models', exist_ok=True)

# Read command-line arguments
opts = sys.argv[1::2]
args = sys.argv[2::2]

# Default parameters
l_ce = 1.0
l2_reg = 10.0
mul = 4.0
TRAIN_BATCH_SIZE = 128 #Changed from 64 to 128 #CHANGE MADE
Feps = 8.0
B_val = 4.0
MAX_EPOCHS = 50
lr_factor = 10.0
EXP_NAME = "default_experiment"
LOG_FILE_NAME = os.path.join('log', f'{EXP_NAME}.txt')

log_dir = os.path.dirname(LOG_FILE_NAME)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Parse arguments and log the changes
for i in range(len(opts)):
    opt = opts[i]
    arg = args[i]
    if opt == '-EXP_NAME':
        EXP_NAME = str(arg)
        LOG_FILE_NAME = os.path.join('log', f'{EXP_NAME}.txt')
        print('EXP_NAME:', EXP_NAME)
    elif opt == '-MAX_EPOCHS':
        MAX_EPOCHS = int(arg)
        print('MAX_EPOCHS:', MAX_EPOCHS)
    elif opt == '-l_ce':
        l_ce = float(arg)
        print('l_ce:', l_ce)
    elif opt == '-B_val':
        B_val = float(arg)
        print('Initial Noise Magnitude:', B_val)
    elif opt == '-l2_reg':
        l2_reg = float(arg)
        print('l2_reg:', l2_reg)
    elif opt == '-b_size':
        TRAIN_BATCH_SIZE = int(arg)
        print('Training Batch Size:', TRAIN_BATCH_SIZE)
    elif opt == '-Feps':
        Feps = float(arg)
        print('RFGSM Epsilon:', Feps)
    elif opt == '-lr_factor':
        lr_factor = float(arg)
        print('lr_factor:', lr_factor)
    elif opt == '-mul':
        mul = float(arg)
        print('Step Mult. factor:', mul)

# Set up logging to the log file
logging.basicConfig(
    filename=LOG_FILE_NAME,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Log experiment details
logging.info(f"Experiment Name: {EXP_NAME}")
logging.info(f"MAX_EPOCHS: {MAX_EPOCHS}")
logging.info(f"l_ce: {l_ce}")
logging.info(f"B_val (Initial Noise Magnitude): {B_val}")
logging.info(f"l2_reg: {l2_reg}")
logging.info(f"TRAIN_BATCH_SIZE: {TRAIN_BATCH_SIZE}")
logging.info(f"Feps (RFGSM Epsilon): {Feps}")
logging.info(f"lr_factor: {lr_factor}")
logging.info(f"mul (Step Multiplier Factor): {mul}")

# You can replace `writer.add_scalar(...)` calls with `logging.info(...)` or other logging calls as needed.
# Example:
logging.info("Experiment setup complete and ready to run.")

In [ ]:
###################################### Function Definitions #######################################

def Guided_Attack(model,loss,image,target,eps=8/255,bounds=[0,1],steps=1,P_out=[],l2_reg=10,alt=1):
    tar = Variable(target.cuda())
    img = image.cuda()
    eps = eps/steps
    for step in range(steps):
        img = Variable(img,requires_grad=True)
        if img.grad is not None:
          img.grad.zero_()
        out  = model(img)
        R_out = nn.Softmax(dim=1)(out)
        cost = loss(out,tar) + alt*l2_reg*(((P_out - R_out)**2.0).sum(1)).mean(0)
        cost.backward()
        per = eps * torch.sign(img.grad.data)
        adv = img.data + per.cuda()
        img = torch.clamp(adv,bounds[0],bounds[1])
    return img


#######################################Cudnn##############################################
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark=True
print('Cudnn status:',torch.backends.cudnn.enabled)
#######################################Set tensor to CUDA#########################################
torch.set_default_tensor_type('torch.cuda.FloatTensor')
#######################################Parameters##################################################
TRAIN_BATCH_SIZE = TRAIN_BATCH_SIZE
VAL_BATCH_SIZE   = 128
TEST_BATCH_SIZE   = 128
BASE_LR          = 0.05 #Change made from 1e-1 to 0.05 #CHANGE MADE
MAX_ITER         = (MAX_EPOCHS*50000)/TRAIN_BATCH_SIZE
MODEL_PREFIX     = 'models/' + EXP_NAME + '_'

####################################### load network ################################################
model = ResNet18()
model.cuda()
model.train()

###################################### load data ####################################################
transform_train = transforms.Compose([
        transforms.RandomCrop(size=32,padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),])

transform_test = transforms.Compose([
        transforms.ToTensor(),])

train_set  = torchvision.datasets.CIFAR10(root='./data', train=True , download=True, transform=transform_train)
val_set    = torchvision.datasets.CIFAR10(root='./data', train=True , download=True, transform=transform_test)
test_set   = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Creating Train-Val split from original Training set
train_size = 49000
valid_size = 1000
test_size  = 10000

train_indices = list(range(50000))
val_indices = []
count = np.zeros(10)
for index in range(len(train_set)):
    _, target = train_set[index]
    if(np.all(count==100)):
        break
    if(count[target]<100):
        count[target] += 1
        val_indices.append(index)
        train_indices.remove(index)

print("Size of train set:",len(train_indices))
print("Size of val set:",len(val_indices))

#get data loader for train, val and test
train_loader = torch.utils.data.DataLoader(train_set,batch_size=TRAIN_BATCH_SIZE ,sampler=SubsetRandomSampler(train_indices))
val_loader   = torch.utils.data.DataLoader(val_set,sampler = SubsetRandomSampler(val_indices),batch_size=VAL_BATCH_SIZE)
test_loader   = torch.utils.data.DataLoader(test_set,batch_size=TEST_BATCH_SIZE)
print('CIFAR10 dataloader: Done')

###################################################################################################
epochs    = MAX_EPOCHS
iteration = 0
loss      = nn.CrossEntropyLoss()
LR = BASE_LR
optimizer = torch.optim.SGD(model.parameters(), lr=LR,momentum=0.9,weight_decay=5e-4)
optimizer.zero_grad()

##################################################################################################
for epoch in range(epochs):
    start = time.time()
    iter_loss =0
    counter =0
    total_ce_loss = torch.tensor([0.0])
    total_reg_loss = torch.tensor([0.0])

    for data, target in train_loader:
        data   = Variable(data).cuda()
        target = Variable(target).cuda()

        out  = model(data)
        P_out = nn.Softmax(dim=1)(out)

        adv_data = data + ((B_val/255.0)*torch.sign(torch.tensor([0.5]) - torch.rand_like(data)).cuda())
        adv_data = torch.clamp(adv_data,0.0,1.0)

        model.eval()
        adv_data = Guided_Attack(model,loss,adv_data,target,eps=Feps/255.0,steps=1,P_out=P_out,l2_reg=l2_reg,alt=(counter%2))

        delta = adv_data - data
        delta = torch.clamp(delta,-8.0/255.0,8.0/255)
        adv_data = data+delta
        adv_data = torch.clamp(adv_data,0.0,1.0)

        model.train()
        optimizer.zero_grad()
        adv_out  = model(adv_data)
        out  = model(data)

        Q_out = nn.Softmax(dim=1)(adv_out)
        P_out = nn.Softmax(dim=1)(out)

        '''LOSS COMPUTATION'''
        closs = loss(out,target)
        reg_loss = ((P_out - Q_out)**2.0).sum(1).mean(0)
        cost = l_ce*closs + l2_reg*reg_loss
        cost.backward()
        optimizer.step()

        total_ce_loss += closs.data
        total_reg_loss += l2_reg*reg_loss.data

        if iteration%100==0:
            msg = 'iter,'+str(iteration)+',clean loss,'+str(closs.data.cpu().numpy()) \
            +',reg loss,'+str(reg_loss.data.cpu().numpy()) \
            +',total loss,'+str(cost.data.cpu().numpy()) \
            +'\n'
            log_file = open(LOG_FILE_NAME,'a+')
            log_file.write(msg)
            log_file.close()
            model.train()

        iteration = iteration + 1
        counter = counter + 1
        sys.stdout.write('\r')
        sys.stdout.write('| Epoch [%3d/%3d] Iter[%3d/%3d] : Loss:%f \t\t'
                %(epoch, MAX_EPOCHS, counter,
                    (train_size/TRAIN_BATCH_SIZE),cost.data.cpu().numpy()))
    end = time.time()
    print('Epoch:',epoch,' Time taken:',(end-start))

    model_name = MODEL_PREFIX+str(epoch)+'.pkl'
    torch.save(model.state_dict(),model_name)

    logging.info(f"Epoch {epoch} - Train CE Loss: {total_ce_loss}")
    logging.info(f"Epoch {epoch} - Train Reg Loss: {total_reg_loss}")

    if epoch in [70,85]:
        LR /= lr_factor
        for param_group in optimizer.param_groups:
            param_group['lr'] = LR
    if epoch == 85:
        l2_reg = l2_reg*mul

#######################################################################################################################
model.eval()

def FGSM_Attack_step(model,loss,image,target,eps=8/255,bounds=[0,1],steps=30):
    tar = Variable(target.cuda())
    img = image.cuda()
    eps = eps/steps
    for step in range(steps):
        img = Variable(img,requires_grad=True)
        if img.grad is not None:
            img.grad.zero_()
        out  = model(img)
        cost = loss(out,tar)
        cost.backward()
        per = eps * torch.sign(img.grad.data)
        adv = img.data + per.cuda()
        img = torch.clamp(adv,bounds[0],bounds[1])
    return img

def MSPGD(model,loss,data,target,eps=8/255,eps_iter=2/255,bounds=[],steps=[7,20,50,100,500]):
    """
    model
    loss : loss used for training
    data : input to network
    target : ground truth label corresponding to data
    eps : perturbation srength added to image
    eps_iter
    """
    if model.training:
        assert 'Model is in  training mode'
    tar = Variable(target.cuda())
    data = data.cuda()
    B,C,H,W = data.size()
    noise  = torch.FloatTensor(np.random.uniform(-eps,eps,(B,C,H,W))).cuda()
    noise  = torch.clamp(noise,-eps,eps)
    img_arr = []
    for step in range(steps[-1]):
        img = data + noise
        img = Variable(img,requires_grad=True)
        if img.grad is not None:
            img.grad.zero_()
        out  = model(img)
        cost = loss(out,tar)
        cost.backward()
        per =  torch.sign(img.grad.data)
        per[:,0,:,:] = (eps_iter * (bounds[0,1] - bounds[0,0])) * per[:,0,:,:]
        if(per.size(1)>1):
            per[:,1,:,:] = (eps_iter * (bounds[1,1] - bounds[1,0])) * per[:,1,:,:]
            per[:,2,:,:] = (eps_iter * (bounds[2,1] - bounds[2,0])) * per[:,2,:,:]
        adv = img.data + per.cuda()
        img.requires_grad =False
        img[:,0,:,:] = torch.clamp(adv[:,0,:,:],bounds[0,0],bounds[0,1])
        if(per.size(1)>1):
            img[:,1,:,:] = torch.clamp(adv[:,1,:,:],bounds[1,0],bounds[1,1])
            img[:,2,:,:] = torch.clamp(adv[:,2,:,:],bounds[2,0],bounds[2,1])
        img = img.data
        noise = img - data
        noise  = torch.clamp(noise,-eps,eps)
        for j in range(len(steps)):
            if step == steps[j]-1:
                img_tmp = data + noise
                img_arr.append(img_tmp)
                break
    return img_arr

def max_margin_loss(x,y):
    B = y.size(0)
    corr = x[range(B),y]
    x_new = x - 1000*torch.eye(10)[y].cuda()
    tar = x[range(B),x_new.argmax(dim=1)]
    loss = tar - corr
    loss = torch.mean(loss)
    return loss

def GAMA_PGD(model, data, target, eps, eps_iter, bounds, steps, w_reg, lin, SCHED, drop):
    #Raise error if in training mode
    if model.training:
        assert 'Model is in  training mode'
    tar = Variable(target.cuda())
    data = data.cuda()
    B,C,H,W = data.size()
    noise  = torch.FloatTensor(np.random.uniform(-eps,eps,(B,C,H,W))).cuda()
    noise  = eps*torch.sign(noise)
    img_arr = []
    W_REG = w_reg
    orig_img = data+noise
    orig_img = Variable(orig_img,requires_grad=True)
    for step in range(steps[-1]):
        # convert data and corresponding into cuda variable
        img = data + noise
        img = Variable(img,requires_grad=True)

        if step in SCHED:
            eps_iter /= drop

        # make gradient of img to zeros using .zero_()
        if img.grad is not None:
            img.grad.zero_()

        # forward pass
        orig_out = model(orig_img)
        P_out = nn.Softmax(dim=1)(orig_out)

        out  = model(img)
        Q_out = nn.Softmax(dim=1)(out)
        #compute loss using true label
        if step <= lin:
            cost =  W_REG*((P_out - Q_out)**2.0).sum(1).mean(0) + max_margin_loss(Q_out,tar)
            W_REG -= w_reg/lin
        else:
            cost = max_margin_loss(Q_out,tar)
        #backward pass
        cost.backward()
        #get gradient of loss wrt data
        per =  torch.sign(img.grad.data)
        #convert eps 0-1 range to per channel range
        per[:,0,:,:] = (eps_iter * (bounds[0,1] - bounds[0,0])) * per[:,0,:,:]
        if(per.size(1)>1):
            per[:,1,:,:] = (eps_iter * (bounds[1,1] - bounds[1,0])) * per[:,1,:,:]
            per[:,2,:,:] = (eps_iter * (bounds[2,1] - bounds[2,0])) * per[:,2,:,:]
        #  ascent
        adv = img.data + per.cuda()
        #clip per channel data out of the range
        img.requires_grad =False
        img[:,0,:,:] = torch.clamp(adv[:,0,:,:],bounds[0,0],bounds[0,1])
        if(per.size(1)>1):
            img[:,1,:,:] = torch.clamp(adv[:,1,:,:],bounds[1,0],bounds[1,1])
            img[:,2,:,:] = torch.clamp(adv[:,2,:,:],bounds[2,0],bounds[2,1])
        img = img.data
        noise = img - data
        noise  = torch.clamp(noise,-eps,eps)

        for j in range(len(steps)):
            if step == steps[j]-1:
                img_tmp = data + noise
                img_arr.append(img_tmp)
                break
    return img_arr

Cudnn status: True
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Size of train set: 49000
Size of val set: 1000
CIFAR10 dataloader: Done
| Epoch [  0/ 50] Iter[383/382] : Loss:1.582171 		Epoch: 0  Time taken: 44.56588268280029
| Epoch [  1/ 50] Iter[383/382] : Loss:1.604499 		Epoch: 1  Time taken: 44.02065086364746
| Epoch [  2/ 50] Iter[383/382] : Loss:1.312807 		Epoch: 2  Time taken: 44.07874393463135
| Epoch [  3/ 50] Iter[383/382] : Loss:1.379934 		Epoch: 3  Time taken: 44.017263650894165
| Epoch [  4/ 50] Iter[383/382] : Loss:1.247452 		Epoch: 4  Time taken: 44.26279950141907
| Epoch [  5/ 50] Iter[383/382] : Loss:1.456808 		Epoch: 5  Time taken: 44.018877267837524
| Epoch [  6/ 50] Iter[383/382] : Loss:1.105199 		Epoch: 6  Time taken: 44.13804650306702
| Epoch [  7/ 50] Iter[383/382] : Loss:1.289301 		Epoch: 7  Time taken: 44.138195753097534
| Epoch [  8/ 50] Iter[383/382] : Loss:1.136593 		Epoch: 8  Time taken: 

In [ ]:
##########################FIND BEST MODEL###############################################
os.makedirs('results', exist_ok=True)
EVAL_LOG_NAME = 'results/'+EXP_NAME+'.txt'
ACC_EPOCH_LOG_NAME = 'results/'+EXP_NAME+'acc_epoch.txt'
ACC_IFGSM_EPOCH_LOG_NAME = 'results/'+EXP_NAME+'ifgsm_acc_epoch.txt'
log_file = open(EVAL_LOG_NAME,'a+')
msg = '##################### iter.FGSM: steps=7,eps=8.0/255####################\n'
log_file.write(msg)
log_file.close()
accuracy_log = np.zeros(MAX_EPOCHS)
for epoch in range(MAX_EPOCHS):
    model_name = MODEL_PREFIX+str(epoch)+'.pkl'
    model.load_state_dict(torch.load(model_name))
    eps=8.0/255
    accuracy = 0
    accuracy_ifgsm = 0
    i = 0
    for data, target in val_loader:
        data   = Variable(data).cuda()
        target = Variable(target).cuda()
        out = model(data)
        prediction = out.data.max(1)[1]
        accuracy = accuracy + prediction.eq(target.data).sum()
        i = i + 1
    for data, target in val_loader:
        data = FGSM_Attack_step(model,loss,data,target,eps=eps,steps=7)
        data   = Variable(data).cuda()
        target = Variable(target).cuda()
        out = model(data)
        prediction = out.data.max(1)[1]
        accuracy_ifgsm = accuracy_ifgsm + prediction.eq(target.data).sum()
    acc = (accuracy.item()*1.0) / (i*VAL_BATCH_SIZE) * 100
    acc_ifgsm = (accuracy_ifgsm.item()*1.0) / (i*VAL_BATCH_SIZE) * 100
    #log accuracy to file
    msg= str(epoch)+','+str(acc)+'\n'
    log_file = open(ACC_EPOCH_LOG_NAME,'a+')
    log_file.write(msg)
    log_file.close()
    logging.info(f"Epoch {epoch}: Clean Validation Acc: {acc}")


    msg1= str(epoch)+','+str(acc_ifgsm)+'\n'
    log_file = open(ACC_IFGSM_EPOCH_LOG_NAME,'a+')
    log_file.write(msg1)
    log_file.close()
    logging.info(f"Epoch {epoch}: IFGSM-7 Validation Acc: {acc_ifgsm}")

    accuracy_log[epoch] = acc_ifgsm
    sys.stdout.write('\r')
    sys.stdout.write('| Epoch [%3d/%3d] : Acc:%f \t\t'
            %(epoch, MAX_EPOCHS,acc))
    sys.stdout.flush()

log_file = open(EVAL_LOG_NAME,'a+')
msg = 'Epoch,'+str(accuracy_log.argmax())+',Acc,'+str(accuracy_log.max())+'\n'
log_file.write(msg)
log_file.close()

model_name = MODEL_PREFIX+str(accuracy_log.argmax())+'.pkl'
model.load_state_dict(torch.load(model_name))
model.eval()
model.cuda()
##################################### FGSM #############################################
EVAL_LOG_NAME = 'results/'+EXP_NAME+'.txt'
log_file = open(EVAL_LOG_NAME,'a+')
msg = '##################### FGSM ####################\n'
log_file.write(msg)
log_file.close()
for eps in np.arange(0.0/255,10.0/255,2.0/255):
    i = 0
    accuracy = 0
    for data, target in test_loader:
        adv = FGSM_Attack_step(model,loss,data,target,eps=eps,steps=1)
        data   = Variable(adv).cuda()
        target = Variable(target).cuda()
        out = model(data)
        prediction = out.data.max(1)[1]
        accuracy = accuracy + prediction.eq(target.data).sum()
        i = i + 1
    acc = (accuracy.item()*1.0) / (test_size) * 100
    log_file = open(EVAL_LOG_NAME,'a+')
    msg = 'eps,'+str(eps)+',Acc,'+str(acc)+'\n'
    log_file.write(msg)
    log_file.close()
##################################### iFGSM #############################################
log_file = open(EVAL_LOG_NAME,'a+')
msg = '##################### iFGSM: step=7 ####################\n'
log_file.write(msg)
log_file.close()
for eps in np.arange(2.0/255,10.0/255,2.0/255):
    i = 0
    accuracy = 0
    for data, target in test_loader:
        adv = FGSM_Attack_step(model,loss,data,target,eps=eps,steps=7)
        data   = Variable(adv).cuda()
        target = Variable(target).cuda()
        out = model(data)
        prediction = out.data.max(1)[1]
        accuracy = accuracy + prediction.eq(target.data).sum()
        i = i + 1
    acc = (accuracy.item()*1.0) / (test_size) * 100
    log_file = open(EVAL_LOG_NAME,'a+')
    msg = 'eps,'+str(eps)+',Acc,'+str(acc)+'\n'
    log_file.write(msg)
    log_file.close()


##################################### PGD, steps=[7,20,50,100,500] #############################################
log_file = open(EVAL_LOG_NAME,'a+')
msg = '##################### PGD: steps=[7,20,50,100,500],eps_iter=2/255 ####################\n'
log_file.write(msg)
log_file.close()
all_steps = [7,20,50,100,500]
num_steps = len(all_steps)
eps = 8.0/255
i = 0
acc_arr = torch.zeros((num_steps))
for data, target in test_loader:
    adv_arr = MSPGD(model,loss,data,target,eps=eps,eps_iter=2.0/255,bounds=np.array([[0,1],[0,1],[0,1]]),steps=all_steps)
    target = Variable(target).cuda()
    for j in range(num_steps):
        data   = Variable(adv_arr[j]).cuda()
        out = model(data)
        prediction = out.data.max(1)[1]
        acc_arr[j] = acc_arr[j] + prediction.eq(target.data).sum()
    i = i + 1
print(acc_arr)
for j in range(num_steps):
    acc_arr[j] = (acc_arr[j].item()*1.0) / (test_size) * 100
    log_file = open(EVAL_LOG_NAME,'a+')
    msg = 'eps,'+str(eps)+',steps,'+str(all_steps[j])+',Acc,'+str(acc_arr[j])+'\n'
    log_file.write(msg)
    log_file.close()


SCHED = [60,85]
drop = 10
lin = 25
w_reg = 50
##################################### GAMA PGD, steps=[60,85,90,100] #############################################
log_file = open(EVAL_LOG_NAME,'a+')
msg = '##################### Gama-PGD Wreg50 lin25, drop by 10 at [60,85]: steps=[60,85,90,100], eps_iter_init=16/255  ####################\n'
log_file.write(msg)
log_file.close()
all_steps = [60,85,90,100]
num_steps = len(all_steps)
eps = 8.0/255
i = 0
acc_arr = torch.zeros((num_steps))
for data, target in test_loader:
    adv_arr = GAMA_PGD(model,data,target,eps=eps,eps_iter=16/255,bounds=np.array([[0,1],[0,1],[0,1]]),steps=all_steps,w_reg=w_reg,lin=lin,SCHED=SCHED,drop=drop)
    target = Variable(target).cuda()
    for j in range(num_steps):
        data   = Variable(adv_arr[j]).cuda()
        out = model(data)
        prediction = out.data.max(1)[1]
        acc_arr[j] = acc_arr[j] + prediction.eq(target.data).sum()
    i = i + 1
print(acc_arr)
for j in range(num_steps):
    acc_arr[j] = (acc_arr[j].item()*1.0) / (test_size) * 100
    log_file = open(EVAL_LOG_NAME,'a+')
    msg = 'eps,'+str(eps)+',steps,'+str(all_steps[j])+',Acc,'+str(acc_arr[j])+'\n'
    log_file.write(msg)
    log_file.close()

<ipython-input-8-eee7d28eb26b>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_name))


| Epoch [ 49/ 50] : Acc:79.003906 		

<ipython-input-8-eee7d28eb26b>:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_name))


tensor([4537., 4351., 4341., 4339., 4341.])
tensor([4153., 3962., 3962., 3961.])
